# Phân tích dữ liệu nhân viên

Notebook này sử dụng module `data_processor` để xử lý dữ liệu nhân viên từ file Excel và tạo báo cáo.

In [1]:
import sys
import os

# Thêm đường dẫn module vào sys.path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

from data_processor import read_excel_file, save_to_excel, merge_dataframes
from utils import create_summary_report, save_json_report, create_directory_if_not_exists
import pandas as pd
import numpy as np
from datetime import datetime

In [23]:
# Đọc dữ liệu nhân viên
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'employees.xlsx')
employees_df = read_excel_file(data_path)

print(f"Đã tải {len(employees_df)} bản ghi nhân viên")
print("\nThông tin cột:")
print(employees_df.info())
print("\n5 dòng đầu:")
employees_df.head()

✅ Đã đọc thành công file: employees.xlsx
📊 Số dòng: 20, Số cột: 6
Đã tải 20 bản ghi nhân viên

Thông tin cột:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            20 non-null     int64         
 1   Họ tên        20 non-null     object        
 2   Phòng ban     20 non-null     object        
 3   Lương cơ bản  20 non-null     int64         
 4   Ngày vào làm  20 non-null     datetime64[ns]
 5   Tuổi          20 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 1.1+ KB
None

5 dòng đầu:


,ID,Họ tên,Phòng ban,Lương cơ bản,Ngày vào làm,Tuổi
0,1,Nguyễn Văn An,Marketing,17784633,2020-01-01 00:00:00.000,23
1,2,Trần Thị Bình,Bán hàng,13664789,2020-03-17 20:12:37.895,41
2,3,Lê Văn Cường,Nhân sự,17695979,2020-06-02 16:25:15.789,49
3,4,Phạm Thị Dung,Bán hàng,17305648,2020-08-18 12:37:53.684,28
4,5,Hoàng Văn Em,Bán hàng,13743066,2020-11-03 08:50:31.579,29


In [24]:
# Phân tích dữ liệu
print("=== PHÂN TÍCH DỮ LIỆU NHÂN VIÊN ===")

# Thống kê theo phòng ban
dept_stats = employees_df.groupby('Phòng ban').agg({
    'ID': 'count',
    'Lương cơ bản': ['mean', 'min', 'max'],
    'Tuổi': 'mean'
}).round(2)

# Flatten MultiIndex columns
dept_stats.columns = ['_'.join(col).strip() for col in dept_stats.columns]

print("\nThống kê theo phòng ban:")
print(dept_stats)

# Thống kê lương
print(f"\nLương trung bình: {employees_df['Lương cơ bản'].mean():,.0f} VND")
print(f"Lương cao nhất: {employees_df['Lương cơ bản'].max():,.0f} VND")
print(f"Lương thấp nhất: {employees_df['Lương cơ bản'].min():,.0f} VND")

# Phân bố tuổi
age_stats = employees_df['Tuổi'].describe()
print(f"\nThống kê tuổi:")
print(age_stats)

=== PHÂN TÍCH DỮ LIỆU NHÂN VIÊN ===

Thống kê theo phòng ban:
           ID_count  Lương cơ bản_mean  Lương cơ bản_min  Lương cơ bản_max  \
Phòng ban                                                                    
Bán hàng          6        16570423.33          10663046          24861870   
IT                1         9465689.00           9465689           9465689   
Kế toán           3        16328953.67          11385357          18959014   
Marketing         5        20611462.00          13147581          24845364   
Nhân sự           5        16824903.40          10428388          21237723   

           Tuổi_mean  
Phòng ban             
Bán hàng       33.33  
IT             29.00  
Kế toán        39.00  
Marketing      33.20  
Nhân sự        37.00  

Lương trung bình: 17,252,846 VND
Lương cao nhất: 24,861,870 VND
Lương thấp nhất: 9,465,689 VND

Thống kê tuổi:
count    20.000000
mean     34.850000
std       9.189668
min      23.000000
25%      27.750000
50%      35.000000
75% 

In [25]:
# Tạo báo cáo tóm tắt
summary_data = {
    'report_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_employees': len(employees_df),
    'departments': employees_df['Phòng ban'].nunique(),
    'average_salary': round(employees_df['Lương cơ bản'].mean(), 2),
    'average_age': round(employees_df['Tuổi'].mean(), 1),
    'salary_range': {
        'min': float(employees_df['Lương cơ bản'].min()),
        'max': float(employees_df['Lương cơ bản'].max())
    },
    'department_breakdown': employees_df['Phòng ban'].value_counts().to_dict(),
    'age_distribution': {
        'under_30': len(employees_df[employees_df['Tuổi'] < 30]),
        'between_30_40': len(employees_df[(employees_df['Tuổi'] >= 30) & (employees_df['Tuổi'] < 40)]),
        'over_40': len(employees_df[employees_df['Tuổi'] >= 40])
    }
}

print("Báo cáo tóm tắt:")
print(f"- Tổng số nhân viên: {summary_data['total_employees']}")
print(f"- Số phòng ban: {summary_data['departments']}")
print(f"- Lương trung bình: {summary_data['average_salary']:,.0f} VND")
print(f"- Tuổi trung bình: {summary_data['average_age']} tuổi")

Báo cáo tóm tắt:
- Tổng số nhân viên: 20
- Số phòng ban: 5
- Lương trung bình: 17,252,846 VND
- Tuổi trung bình: 34.8 tuổi


In [26]:
# Lưu kết quả ra thư mục output
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'output')

# 1. Lưu báo cáo thống kê phòng ban thành Excel
stats_file = os.path.join(output_dir, 'employee_department_stats.xlsx')
save_to_excel(dept_stats.reset_index(), stats_file)
print(f"✅ Đã lưu thống kê phòng ban: {stats_file}")

# 2. Lưu dữ liệu tóm tắt thành JSON
json_file = os.path.join(output_dir, 'employee_summary.json')
save_json_report(summary_data, json_file)
print(f"✅ Đã lưu tóm tắt JSON: {json_file}")

# 3. Lưu báo cáo văn bản
report_file = os.path.join(output_dir, 'employee_analysis_report.txt')
create_summary_report(summary_data, report_file)
print(f"✅ Đã lưu báo cáo văn bản: {report_file}")

# 4. Lưu danh sách nhân viên lương cao (top 20%)
high_salary_threshold = employees_df['Lương cơ bản'].quantile(0.8)
high_salary_df = employees_df[employees_df['Lương cơ bản'] > high_salary_threshold]
high_salary_file = os.path.join(output_dir, 'high_salary_employees.xlsx')
save_to_excel(high_salary_df, high_salary_file)
print(f"✅ Đã lưu danh sách lương cao: {high_salary_file}")

# 5. Lưu nhân viên trẻ (dưới 30 tuổi)
young_employees = employees_df[employees_df['Tuổi'] < 30]
young_file = os.path.join(output_dir, 'young_employees.xlsx')
save_to_excel(young_employees, young_file)
print(f"✅ Đã lưu danh sách nhân viên trẻ: {young_file}")

print(f"\n🎉 Hoàn thành! Đã tạo {5} file output từ phân tích dữ liệu nhân viên.")

💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\employee_department_stats.xlsx
✅ Đã lưu thống kê phòng ban: d:\twan-projects\nbrunner\resources\output\employee_department_stats.xlsx
💾 Đã lưu JSON tại: d:\twan-projects\nbrunner\resources\output\employee_summary.json
✅ Đã lưu tóm tắt JSON: d:\twan-projects\nbrunner\resources\output\employee_summary.json
📊 Đã tạo báo cáo tại: d:\twan-projects\nbrunner\resources\output\employee_analysis_report.txt
✅ Đã lưu báo cáo văn bản: d:\twan-projects\nbrunner\resources\output\employee_analysis_report.txt
💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\high_salary_employees.xlsx
✅ Đã lưu danh sách lương cao: d:\twan-projects\nbrunner\resources\output\high_salary_employees.xlsx
💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\young_employees.xlsx
✅ Đã lưu danh sách nhân viên trẻ: d:\twan-projects\nbrunner\resources\output\young_employees.xlsx

🎉 Hoàn thành! Đã tạo 5 file output từ phân tích dữ liệu nhân vi

In [ ]:
import time
time.sleep(5)  # Đợi 1 giây để đảm bảo mọi thứ đã hoàn thành